<a href="https://colab.research.google.com/github/thedabbler24/gw-mem-analysis/blob/main/gw150914.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install bilby gwmemory gwpy gwosc matplotlib

In [ ]:
# Relevant packages
import gwmemory
import matplotlib.pyplot as plt
import numpy as np
from gwosc import datasets
from gwpy.timeseries import TimeSeries
from gwosc.datasets import event_gps
import bilby
import logging
from bilby.core.utils.random import seed
import warnings
import lal

In [ ]:
seed(123)
# Setting up logger
bilby.core.utils.setup_logger(logging.INFO)

# Setting the result directory
outdir = 'outdir_gw150914'
label = 'GW150914_standard'

warnings.filterwarnings("ignore", message="Cannot convert from zenith/azimuth")
warnings.filterwarnings("ignore", message="Cannot find .*_time in parameters")

In [ ]:
# Setting up empty interferometer
H1 = bilby.gw.detector.get_empty_interferometer('H1')

# Setting up data
event = "GW150914"
gps_time = event_gps(event)

detector = "H1"
sampling_freq = 4096
time_series = TimeSeries.fetch_open_data(detector,gps_time - 2,gps_time + 2,sample_rate=sampling_freq,cache = True, dataset=event)

strain_data = time_series.value
time = time_series.times.value
H1.set_strain_data_from_gwpy_timeseries(time_series = time_series)

#plt.plot(time,strain_data)
#plt.xlabel("Time (s)")
#plt.ylabel("Strain")
#plt.show()


In [ ]:
# Defining parameters of GW150914
parameters = dict(q=1.2,spin_1 = [0,0,0.3],spin_2 = [0,0,0.2],total_mass = 63.1,distance = 440.0, inc = 2.8, phase = 0.0, geocent_time = gps_time)

# Setting priors
priors = bilby.gw.prior.BBHPriorDict()

for key in ["chirp_mass", "mass_ratio","phi_12","phi_jl"]:
    priors.pop(key, None)

priors['mass_1'] = bilby.core.prior.Uniform(name='mass_1', minimum=20, maximum=80)
priors['mass_2'] = bilby.core.prior.Uniform(name='mass_2', minimum=10, maximum=60)
priors["geocent_time"] = bilby.core.prior.Uniform(gps_time - 0.1, gps_time + 0.1, name="geocent_time", unit="s")
priors["tilt_1"] = bilby.core.prior.DeltaFunction(0, name="tilt_1")
priors["tilt_2"] = bilby.core.prior.DeltaFunction(0, name="tilt_2")

#print("Priors keys:", list(priors.keys()))

In [ ]:
# Waveform generator
waveform_arguments = dict(waveform_approximant = "IMRPhenomD",
                          reference_frequency = 50.0,
                          minimum_frequency = 20.0)

waveform_generator = bilby.gw.waveform_generator.WaveformGenerator(
    duration = 4.0,
    sampling_frequency = sampling_freq,
    frequency_domain_source_model = bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments = waveform_arguments
)


In [ ]:
# Defining Likelihood
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers = [H1],
    waveform_generator = waveform_generator,
    priors = priors,
    reference_frame = "sky",
    time_reference = "geocent"
)

likelihood.parameters = {key: None for key in priors.keys()}
#print("Likelihood parameters:", likelihood.parameters)




In [ ]:
# Sampler
result = bilby.run_sampler(
    likelihood = likelihood,
    priors = priors,
    sampler = "dynesty",
    nlive = 500,
    max_n_steps = 5000,
    npool = 1,
    dlogz = 0.01
    outdir = 'outdir_gw150914',
    label = "GW150914_standard",
    resume = True,
    clean = True
)

result.plot_corner()              # Posterior distributions
result.plot_waveform_posterior()  # Overlay waveforms on data
print(result.log_evidence)        # Bayesian evidence